In [ ]:
import pandas as pd
import plotly.express as px
import json
from datetime import datetime

# Path to your log file
log_file_path = 'server3.log'

# Threshold values for small models
prompt_eval_time_small_model_threshold = 10
generation_eval_time_small_model_threshold = 100

# Read and parse log file with categorization based on token time performance
def read_and_parse_logs(path):
    small_models = []
    large_models = []
    with open(path, 'r') as file:
        for line in file:
            try:
                log_entry = json.loads(line)
                if log_entry.get('function') == 'print_timings':
                    timestamp = datetime.fromtimestamp(log_entry['timestamp'])
                    
                    def process_token_type(token_type, token_key, threshold, data_list, model_lists):
                        if token_key in log_entry and 't_token' in log_entry and log_entry['t_token'] is not None:
                            model_type = 'small' if log_entry['t_token'] < threshold else 'large'
                            data_entry = {'date': timestamp.date(), 'tokens': log_entry[token_key], 'type': token_type, 'model': model_type}
                            if model_type == 'small':
                                model_lists[0].append(data_entry)
                            else:
                                model_lists[1].append(data_entry)

                    process_token_type('Prompt Tokens', 'n_prompt_tokens_processed', prompt_eval_time_small_model_threshold, small_models, (small_models, large_models))
                    process_token_type('Generation Tokens', 'n_decoded', generation_eval_time_small_model_threshold, small_models, (small_models, large_models))
                        
            except json.JSONDecodeError:
                continue

    return small_models, large_models



# Running the parsing function
small_models, large_models = read_and_parse_logs(log_file_path)

# Convert lists to DataFrame
df_small = pd.DataFrame(small_models)
df_large = pd.DataFrame(large_models)

# Group by date, type and model category
df_small_summary = df_small.groupby(['date', 'type']).sum().reset_index()
df_large_summary = df_large.groupby(['date', 'type']).sum().reset_index()

# Concatenate dataframes for bar chart
df_summary = pd.concat([df_small_summary.assign(Model='Small Models'), 
                        df_large_summary.assign(Model='Large Models')])

# Plot using Plotly
fig = px.bar(df_summary, x='date', y='tokens', color='type', barmode='group', facet_col='Model', title="Tokens Processed Over Time by Model Size")
fig.update_layout(xaxis_title='Date', yaxis_title='Number of Tokens', legend_title='Token Type / Model Size')
fig.show()
